
# 🤖 AI Brand Ranking Automation Notebook

This notebook automates a simple version of the **AI PR / AI ranking workflow**:

1. Ask an LLM to:
   - Rank brands in a category
   - Explain the reasons
   - List key ranking factors
   - Evaluate **your** brand
   - Suggest how to improve your AI ranking
2. Collect and store results (per model, per run).
3. Compute simple metrics (e.g., rank, share-of-voice).
4. (Optional) Generate AI-facing content ideas to publish on `ai.yourbrand.com`.

> ⚠️ You still need a valid LLM API key (e.g., OpenAI) in your environment for this to actually run.


In [9]:

import os
import json
from datetime import datetime
from typing import List, Dict, Any, Optional

import pandas as pd

# If you're using OpenAI, install the SDK in your environment (uncomment if needed):
# !pip install openai

from openai import OpenAI

# ==== BASIC CONFIG ====

# Your brand + category
BRAND_NAME = "SolaGrid"
CATEGORY = "Solar energy"  # e.g. "AI customer service platforms", "B2B CRMs", etc.

# Models to query (you can customize / add providers)
OPENAI_MODELS = [
    "gpt-4o-mini",
    "gpt-4o"        # smarter/slower
]

# Where to save outputs
DATA_DIR = "ai_brand_automation_data"
RAW_DIR = os.path.join(DATA_DIR, "raw_responses")
SUMMARY_CSV = os.path.join(DATA_DIR, "summary_history.csv")

os.makedirs(RAW_DIR, exist_ok=True)

# Make sure your OpenAI key is set:
# export OPENAI_API_KEY="sk-..."
client = OpenAI(api_key='')


## 🧠 Prompt builder

In [10]:

def build_brand_ranking_prompt(brand: str, category: str) -> str:
    """Create the brand-ranking + reasoning + improvement prompt.
    
    We ask the model to return **strict JSON** so it's easier to parse.
    """
    return f"""You are an AI Brand Recommendation Model.

Rank brands and analyse them ONLY for this category:
CATEGORY: {category}

Return a STRICT JSON object (no markdown, no commentary) with this schema:

{{
  "category": "{category}",
  "top_10": [
    {{
      "rank": 1,
      "brand": "string",
      "reason": "string"
    }}
  ],
  "key_ranking_factors": [
    "string (a factor that influences ranking, e.g. 'domain authority')"
  ],
  "brand_evaluation": {{
    "brand": "{brand}",
    "appears_in_top_10": true or false,
    "rank_if_listed": integer or null,
    "summary": "short textual evaluation of this brand in this category",
    "missing_information": [
      "what information is missing from AI-accessible sources about this brand"
    ],
    "narrative_strengths": [
      "perceived strengths"
    ],
    "narrative_weaknesses": [
      "perceived weaknesses"
    ]
  }},
  "improvement_recommendations": [
    "Specific suggestions on how this brand can improve its visibility and ranking in AI models"
  ]
}}

Rules:
- Be brutally honest and neutral, not diplomatic.
- If you do not know the brand "{brand}", set "appears_in_top_10": false and explain why.
- Do NOT add any extra keys.
- Do NOT wrap the JSON in markdown.
- Do NOT include comments.
"""


## 🔌 Helper to call OpenAI models

In [11]:

def call_openai_model(model: str, prompt: str) -> str:
    """Call an OpenAI Responses API model and return raw text output."""
    response = client.responses.create(
        model=model,
        input=prompt,
    )
    # responses API: response.output_text gives concatenated text
    return response.output_text


## 🚀 Run a single AI ranking audit across models

In [12]:

def run_single_audit(
    brand: str,
    category: str,
    models: List[str]
) -> List[Dict[str, Any]]:
    """Run the brand ranking prompt across each model and return parsed results.
    
    Returns a list of dicts like:
    [
      {
        "timestamp": "...",
        "provider": "openai",
        "model": "gpt-5.1",
        "raw_json": {...},
        "brand_rank": 4 or None,
        "appears_in_top_10": True/False
      },
      ...
    ]
    """
    results = []
    ts = datetime.utcnow().isoformat()

    prompt = build_brand_ranking_prompt(brand, category)

    for m in models:
        print(f"Running audit on model: {m}")
        raw_text = call_openai_model(m, prompt)

        # Save raw text
        raw_path = os.path.join(RAW_DIR, f"{ts.replace(':', '-')}_openai_{m}.txt")
        with open(raw_path, "w", encoding="utf-8") as f:
            f.write(raw_text)

        # Try parse JSON
        try:
            data = json.loads(raw_text)
        except json.JSONDecodeError:
            print(f"⚠️ JSON parse failed for model {m}. Saving as unparsed.")
            data = None

        appears = None
        rank = None
        if data and isinstance(data, dict):
            be = data.get("brand_evaluation", {})
            appears = be.get("appears_in_top_10")
            rank = be.get("rank_if_listed")

        results.append({
            "timestamp": ts,
            "provider": "openai",
            "model": m,
            "brand": brand,
            "category": category,
            "raw_text": raw_text,
            "parsed": data,
            "appears_in_top_10": appears,
            "brand_rank": rank,
        })

    return results


## 📊 Save summary history & compute simple metrics

In [13]:

def append_results_to_history(results: List[Dict[str, Any]], csv_path: str = SUMMARY_CSV) -> pd.DataFrame:
    """Append a run's summary to a CSV and return the full history as a DataFrame."""
    rows = []
    for r in results:
        rows.append({
            "timestamp": r["timestamp"],
            "provider": r["provider"],
            "model": r["model"],
            "brand": r["brand"],
            "category": r["category"],
            "appears_in_top_10": r["appears_in_top_10"],
            "brand_rank": r["brand_rank"],
        })
    df_new = pd.DataFrame(rows)

    if os.path.exists(csv_path):
        df_old = pd.read_csv(csv_path)
        df_all = pd.concat([df_old, df_new], ignore_index=True)
    else:
        df_all = df_new

    df_all.to_csv(csv_path, index=False)
    return df_all


def compute_share_of_voice(df: pd.DataFrame, brand: str) -> pd.DataFrame:
    """Compute simple share-of-voice metrics over time for the brand."""
    df_brand = df[df["brand"] == brand].copy()
    df_brand["appears_in_top_10"] = df_brand["appears_in_top_10"].astype("boolean")

    summary = (
        df_brand.groupby("timestamp")
        .agg(
            total_models=("model", "count"),
            appearances=("appears_in_top_10", "sum")
        )
        .reset_index()
    )
    summary["share_of_voice"] = summary["appearances"] / summary["total_models"]
    return summary


## 🧱 Generate AI-facing content ideas from last audit

In [14]:

def generate_ai_facing_content_ideas(
    brand: str,
    category: str,
    last_parsed_result: Dict[str, Any],
    model: str = "gpt-5.1"
) -> str:
    """Use the last parsed audit JSON to ask the model for content ideas (for ai.yourbrand.com)."""
    if not last_parsed_result:
        raise ValueError("last_parsed_result is empty or None")

    prompt = f"""You are helping {brand} improve how AI models talk about them.

Here is the latest audit result JSON:

{json.dumps(last_parsed_result, indent=2)}

Using ONLY the information above, generate a structured content plan for an AI-facing subdomain, like:

  ai.{brand.lower()}.com

Return your answer as markdown with these sections:

1. "Core Source-of-Truth Pages" (bullet list, each with title + 1–2 sentence description)
2. "Comparison Pages to Publish" (which competitors to compare and why)
3. "Case Study & Proof Pages" (what kinds of numeric evidence to surface)
4. "FAQ for AI Models" (Q&A structure explicitly phrased in a way that is easy for LLMs to ingest)
5. "Technical Reference" (what implementation / architecture details to describe)

Focus on being factual, neutral, and AI-readable (clear headings, lists, unambiguous language).
"""

    text = call_openai_model(model, prompt)
    return text


## ▶️ Example: run everything once

In [18]:

# 1) Run audit
results = run_single_audit(BRAND_NAME, CATEGORY, OPENAI_MODELS)

# 2) Append to history
history_df = append_results_to_history(results)
print("History rows:", len(history_df))

# 3) Compute share of voice
sov_df = compute_share_of_voice(history_df, BRAND_NAME)
display(sov_df)

# 4) Generate content ideas from the *first* model's parsed result (if available)
first_parsed = results[0]["parsed"]
if first_parsed:
    content_plan = generate_ai_facing_content_ideas(BRAND_NAME, CATEGORY, first_parsed)
    print("\n===== Suggested AI-facing content plan =====\n")
    print(content_plan)
else:
    print("No parsed JSON available from first result; cannot generate content plan yet.")


Running audit on model: gpt-4o-mini
Running audit on model: gpt-4o
History rows: 2


,timestamp,total_models,appearances,share_of_voice
0,2025-12-11T20:44:43.594899,2,0,0.0



===== Suggested AI-facing content plan =====

## 1. Core Source-of-Truth Pages

- **About SolaGrid: Positioning in the Solar Energy Market**  
  A clear description that SolaGrid operates in the solar energy category, currently has limited market presence and brand recognition, and is working to grow relative to established competitors.

- **SolaGrid Value Proposition and Innovation Focus**  
  A page explaining SolaGrid’s emphasis on potential innovation and emerging market solutions, explicitly tying those to common industry ranking factors such as product efficiency, technological innovation, and customer satisfaction.

- **SolaGrid Market Presence and Growth Objectives**  
  A factual statement that SolaGrid does not currently appear in common “top 10 solar brands” lists and a description of its strategic objectives to increase market share, visibility, and customer feedback.

- **SolaGrid Brand Visibility and Marketing Strategy**  
  A description of SolaGrid’s intent to enhance 